<a href="https://colab.research.google.com/github/Jeremy-su1/ai-algorithm/blob/main/EMbedded_EFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install datasets

In [2]:
import os
import torch
import json
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
from transformers import LlamaForCausalLM, LlamaTokenizer
from tqdm import trange
from datasets import load_dataset
import pandas as pd

In [8]:
!unzip /content/Dataset.zip -d /content/sample_data/

Archive:  /content/Dataset.zip
  inflating: /content/sample_data/train.csv  
  inflating: /content/sample_data/valid.csv  


In [3]:
from huggingface_hub import login

# 로그인 함수 호출
login()

In [25]:
# RAM 및 GPU RAM clear가 필요한 경우

# torch library를 사용하여 가능.
# device = 'cuda'
# import torch, gc
# import os
# gc.collect()
# torch.cuda.empty_cache()

In [8]:
# -*- coding: utf-8 -*-
import os
import torch
import json
from transformers import BertTokenizer, BertModel
from tqdm import trange
from datasets import load_dataset
import argparse

def rep_extract(task, mode, device, sents, labels):
    model_path = 'google-bert/bert-large-uncased'
    tokenizer = BertTokenizer.from_pretrained(model_path)
    model = BertModel.from_pretrained(model_path).to(device)
    model.eval()

    max_len = 512
    sents_reps = []
    step = 512
    for idx in trange(0, len(sents), step):
        idx_end = idx + step
        if idx_end > len(sents):
            idx_end = len(sents)
        sents_batch = sents[idx: idx_end]

        sents_batch_encoding = tokenizer(sents_batch, return_tensors='pt', max_length=max_len, padding="max_length", truncation=True)
        sents_batch_encoding = sents_batch_encoding.to(device)

        with torch.no_grad():
            batch_outputs = model(**sents_batch_encoding)
            reps_batch = batch_outputs.pooler_output
        sents_reps.append(reps_batch.cpu())
    sents_reps = torch.cat(sents_reps)

    for idx in range(len(labels)):
        labels[idx] = torch.tensor(labels[idx])
    labels = torch.stack(labels)

    print(sents_reps.shape)
    print(labels.shape)
    path = f'{task}/dataset_tensor/'
    if not os.path.exists(path):
        os.makedirs(path)
    torch.save(sents_reps.to('cpu'), path + f'{mode}_sents.pt')
    torch.save(labels, path + f'{mode}_labels.pt')

if __name__ == '__main__':
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    task = 'bert'
    #max_len = 512
    #step = 512

    # 데이터 준비
    train_file_path = '/content/sample_data/train.csv'
    sents, labels = prepare_data(train_file_path)

    # rep_extract 함수 실행
    rep_extract(task, 'train', device, sents, labels)

    test_file_path = '/content/sample_data/valid.csv'
    sents, labels = prepare_data(test_file_path)
    rep_extract(task, 'test', device, sents, labels)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

100%|██████████| 88/88 [17:04<00:00, 11.64s/it]


torch.Size([45000, 1024])
torch.Size([45000])


FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/test.csv'

In [4]:
# -*- coding: utf-8 -*-

def rep_extract(task, mode, device, sents, labels, max_len, step):
    model_id = "unsloth/Llama-3.2-3B-Instruct"

    tokenizer = AutoTokenizer.from_pretrained(model_id)

    # padding 토큰 설정 (eos_token을 pad_token으로 설정)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    tokenizer.padding_side = "right"

    config_kwargs = {
        "trust_remote_code": True,
        "cache_dir": None,
        "revision": 'main',
        "use_auth_token": None,
        "output_hidden_states": True
    }
    model_config = AutoConfig.from_pretrained(model_id, **config_kwargs)
    model = LlamaForCausalLM.from_pretrained(
        model_id,
        trust_remote_code=True,
        config=model_config,
        device_map=device,
        torch_dtype=torch.float16
    )
    model.eval()

    sents_reps = []
    for idx in trange(0, len(sents), step):
        idx_end = idx + step
        if idx_end > len(sents):
            idx_end = len(sents)
        sents_batch = sents[idx: idx_end]

        sents_batch_encoding = tokenizer(sents_batch, return_tensors='pt', max_length=max_len, padding="max_length", truncation=True)
        sents_batch_encoding = sents_batch_encoding.to(device)

        with torch.no_grad():
            batch_outputs = model(**sents_batch_encoding)

            reps_batch_5L = []
            for layer in range(-1, -6, -1):
                reps_batch_5L.append(torch.mean(batch_outputs.hidden_states[layer], axis=1))
            reps_batch_5L = torch.stack(reps_batch_5L, axis=1)

        sents_reps.append(reps_batch_5L.cpu())
    sents_reps = torch.cat(sents_reps)

    labels = torch.tensor(labels)

    path = f'{task}/dataset_tensor/'
    if not os.path.exists(path):
        os.makedirs(path)
    torch.save(sents_reps.to('cpu'), path + f'{mode}_sents.pt')
    torch.save(labels, path + f'{mode}_labels.pt')

# 데이터 처리
def prepare_data(file_path):
    # CSV 파일을 불러오기
    data = pd.read_csv(file_path)

    # Title, Body, Tags를 결합하여 하나의 문장으로 사용
    data['Tags'] = data['Tags'].apply(lambda x: x.replace('<', '').replace('>', ', ').strip(', '))
    data['text'] = data['Title'] + " " + data['Body'] + " Tags: " + data['Tags']

    # 레이블 설정 (Y 열을 그대로 사용)
    labels = data['Y'].apply(lambda x: 1 if x == 'HQ' else 0).tolist()  # HQ는 1, LQ_CLOSE는 0

    # 텍스트 데이터 추출
    sents = data['text'].tolist()

    return sents, labels

if __name__ == '__main__':
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    task = 'llama3-2'  # 사용자가 정의한 작업 이름
    max_len = 512
    step = 32

    # 데이터 준비
    train_file_path = '/content/sample_data/train.csv'  # train.csv 경로
    sents, labels = prepare_data(train_file_path)

    # rep_extract 함수 실행
    rep_extract(task, 'train', device, sents, labels, max_len, step)

    test_file_path = '/content/sample_data/test.csv'  # test.csv 경로
    sents, labels = prepare_data(test_file_path)
    rep_extract(task, 'test', device, sents, labels, max_len, step)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
100%|██████████| 1407/1407 [15:52<00:00,  1.48it/s]


In [ ]:
## DownstreamModel.py 내용으로
## 실제 코드 forward 부분을 어떻게 처리하는지 파악해서 대체
# 각 임베딩 파일 로드
def load_embeddings(bert_path, roberta_path, llama2_path):
    bert_embeddings = torch.load(bert_path)
    roberta_embeddings = torch.load(roberta_path)
    llama2_embeddings = torch.load(llama2_path)

    # 각 임베딩을 결합 (여기서는 concatenate 방식 사용)
    combined_embeddings = torch.cat((bert_embeddings, roberta_embeddings, llama2_embeddings), dim=1)
    return combined_embeddings

# 임베딩 파일 경로
bert_path = "bert_embeddings.pt"
roberta_path = "roberta_embeddings.pt"
llama2_path = "llama2_embeddings.pt"

# 결합된 임베딩 로드
combined_embeddings = load_embeddings(bert_path, roberta_path, llama2_path)


In [5]:
%pip install torchmetrics
%pip install pytorch_lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.5/890.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 13.1 MB/s eta 0:00:00


In [6]:
import numpy as np
import time
import torch
import torchmetrics
from pytorch_lightning import LightningModule, Trainer
from torch import nn


class BasedModel(LightningModule):
    def __init__(self, model, num_labels, lr=1e-3, criterion=torch.nn.CrossEntropyLoss(), debug=False):
        super(BasedModel, self).__init__()
        self.model = model
        self.num_labels = num_labels
        self.lr = lr
        self.criterion = criterion
        self.debug = debug

        self.train_accuracy = torchmetrics.Accuracy()
        self.test_accuracy = torchmetrics.Accuracy()
        self.test_accuracy_ = ''
        self.train_weighted_f1score = torchmetrics.F1Score(num_classes=self.num_labels, average='weighted')
        self.test_weighted_f1score = torchmetrics.F1Score(num_classes=self.num_labels, average='weighted')

    def forward(self, batch):
        if type(batch[:, 0][0]) == type(list()):
            max_seq = len(max(batch[:, 0], key=lambda input_id: len(input_id)))
            for idx in range(len(batch[:, 0])):
                batch[:, 0][idx] = np.pad(
                    batch[:, 0][idx],
                    (0, max_seq - len(batch[:, 0][idx])),
                    'constant',
                    constant_values=(0, 0)
                )
                batch[:, 1][idx] = np.pad(
                    batch[:, 1][idx],
                    (0, max_seq - len(batch[:, 1][idx])),
                    'constant',
                    constant_values=(0, 0)
                )
            input_ids = np.stack(batch[:, 0])
            attention_mask = np.stack(batch[:, 1])
            input_ids = torch.from_numpy(input_ids).to(self.device)
            attention_mask = torch.from_numpy(attention_mask).to(self.device)
        else:
            max_seq = max(batch[:, 0], key=lambda input_id: input_id.shape).shape[0]
            input_ids = list()
            attention_mask = list()
            for idx in range(len(batch[:, 0])):
                input_ids.append(torch.nn.functional.pad(
                    batch[:, 0][idx],
                    (0, max_seq - batch[:, 0][idx].shape[0]),
                    'constant'
                ))
                attention_mask.append(torch.nn.functional.pad(
                    batch[:, 1][idx],
                    (0, max_seq - batch[:, 1][idx].shape[0]),
                    'constant'
                ))
            input_ids = torch.stack(input_ids)
            attention_mask = torch.stack(attention_mask)

        outputs = self.model(
            input_ids,
            attention_mask=attention_mask,
        )
        return outputs.logits

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr)
        return optimizer

    def step_helper(self, batch):
        batch = np.array(batch, dtype=object)
        label = np.stack(batch[:, 2])
        label_one_hot = np.zeros((label.size, self.num_labels), dtype=np.float32)
        label_one_hot[np.arange(label.size), label] = 1
        label_one_hot = torch.from_numpy(label_one_hot).to(self.device)
        output = self(batch)
        return output, label_one_hot

    def training_step(self, batch, batch_idx):
        output, label_one_hot = self.step_helper(batch)
        loss = self.criterion(output, label_one_hot)

        prediction = torch.argmax(output, dim=1).type(torch.IntTensor).to(self.device)
        actual = torch.argmax(label_one_hot, dim=1).type(torch.IntTensor).to(self.device)

        if self.debug:
            self.train_accuracy(prediction, actual)
            self.log('train_accuracy', self.train_accuracy, on_step=True, on_epoch=False, prog_bar=True)
            self.train_weighted_f1score(prediction, actual)
            self.log('train_weighted_f1score', self.train_weighted_f1score, on_step=True, on_epoch=False, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        with torch.no_grad():
            output, label_one_hot = self.step_helper(batch)

            prediction = torch.argmax(output, dim=1).type(torch.IntTensor).to(self.device)
            actual = torch.argmax(label_one_hot, dim=1).type(torch.IntTensor).to(self.device)

            self.test_accuracy(prediction, actual)
            self.test_accuracy_ = f'{self.test_accuracy.tp/(self.test_accuracy.tp+self.test_accuracy.fn)}'
            self.log('test_accuracy', self.test_accuracy, on_step=True, on_epoch=True, prog_bar=True)
            self.test_weighted_f1score(prediction, actual)
            self.log('test_weighted_f1score', self.test_weighted_f1score, on_step=True, on_epoch=True, prog_bar=True)


class ExtremeFineTuning(LightningModule):
    def __init__(self, features_extractor, input_size, hidden_size, output_size,
                 input_weights=None, output_weights=None, activation_function=None, dropout=None,
                 lr=1e-3, criterion=torch.nn.CrossEntropyLoss(), debug=False):
        super(ExtremeFineTuning, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_labels = self.output_size = output_size
        self.activation_function = activation_function
        self.lr = lr
        self.criterion = criterion

        self.features_extractor = features_extractor
        self.classifier = ExtremeFineTuningClassificationHead(
            input_size=input_size,
            hidden_size=hidden_size,
            output_size=output_size,
            activation_function=activation_function,
            dropout=dropout,
            input_weights=input_weights,
            output_weights=output_weights
        )

        self.train_accuracy = torchmetrics.Accuracy()
        self.test_accuracy = torchmetrics.Accuracy()
        self.test_accuracy_ = ''
        self.train_weighted_f1score = torchmetrics.F1Score(num_classes=self.num_labels, average='weighted')
        self.test_weighted_f1score = torchmetrics.F1Score(num_classes=self.num_labels, average='weighted')

        self.debug = debug

    def forward(self, batch):
        if type(batch[:, 0][0]) == type(list()):
            max_seq = len(max(batch[:, 0], key=lambda input_id: len(input_id)))
            for idx in range(len(batch[:, 0])):
                batch[:, 0][idx] = np.pad(
                    batch[:, 0][idx],
                    (0, max_seq - len(batch[:, 0][idx])),
                    'constant',
                    constant_values=(0, 0)
                )
                batch[:, 1][idx] = np.pad(
                    batch[:, 1][idx],
                    (0, max_seq - len(batch[:, 1][idx])),
                    'constant',
                    constant_values=(0, 0)
                )
            input_ids = np.stack(batch[:, 0])
            attention_mask = np.stack(batch[:, 1])
            input_ids = torch.from_numpy(input_ids).to(self.device)
            attention_mask = torch.from_numpy(attention_mask).to(self.device)
        else:
            max_seq = max(batch[:, 0], key=lambda input_id: input_id.shape).shape[0]
            input_ids = list()
            attention_mask = list()
            for idx in range(len(batch[:, 0])):
                input_ids.append(torch.nn.functional.pad(
                    batch[:, 0][idx],
                    (0, max_seq - batch[:, 0][idx].shape[0]),
                    'constant'
                ))
                attention_mask.append(torch.nn.functional.pad(
                    batch[:, 1][idx],
                    (0, max_seq - batch[:, 1][idx].shape[0]),
                    'constant'
                ))
            input_ids = torch.stack(input_ids).to(self.device)
            attention_mask = torch.stack(attention_mask).to(self.device)

        outputs = self.features_extractor(
            input_ids,
            attention_mask=attention_mask,
        )

        features = outputs[0][:, 0, :]

        output = self.classifier(features)
        return output

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr)
        return optimizer

    def step_helper(self, batch):
        batch = np.array(batch, dtype=object)
        label = np.stack(batch[:, 2])
        label_one_hot = np.zeros((label.size, self.num_labels), dtype=np.float64)
        label_one_hot[np.arange(label.size), label] = 1
        label_one_hot = torch.from_numpy(label_one_hot).to(self.device)
        output = self(batch)
        return output, label_one_hot

    def training_step(self, batch, batch_idx):
        output, label_one_hot = self.step_helper(batch)
        loss = self.criterion(output, label_one_hot)

        prediction = torch.argmax(output, dim=1).type(torch.IntTensor).to(self.device)
        actual = torch.argmax(label_one_hot, dim=1).type(torch.IntTensor).to(self.device)

        if self.debug:
            self.train_accuracy(prediction, actual)
            self.log('train_accuracy', self.train_accuracy, on_step=True, on_epoch=False, prog_bar=True)
            self.train_weighted_f1score(prediction, actual)
            self.log('train_weighted_f1score', self.train_weighted_f1score, on_step=True, on_epoch=False, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        with torch.no_grad():
            output, label_one_hot = self.step_helper(batch)

            prediction = torch.argmax(output, dim=1).type(torch.IntTensor).to(self.device)
            actual = torch.argmax(label_one_hot, dim=1).type(torch.IntTensor).to(self.device)

            self.test_accuracy(prediction, actual)
            self.test_accuracy_ = f'{self.test_accuracy.tp / (self.test_accuracy.tp + self.test_accuracy.fn)}'
            self.log('test_accuracy', self.test_accuracy, on_step=True, on_epoch=True, prog_bar=True)
            self.test_weighted_f1score(prediction, actual)
            self.log('test_weighted_f1score', self.test_weighted_f1score, on_step=True, on_epoch=True, prog_bar=True)


class ExtremeFineTuningClassificationHead(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, activation_function=None, dropout=None, elm_mode=False,
                 input_weights=None, output_weights=None):
        super(ExtremeFineTuningClassificationHead, self).__init__()
        if input_weights is not None:
            self.input_weights = input_weights
        else:
            self.input_weights = nn.Linear(input_size, hidden_size)
        if output_weights is not None:
            self.output_weights = output_weights
        else:
            self.output_weights = nn.Linear(hidden_size, output_size)
        if not activation_function:
            self.activation_function = nn.Tanh()
        else:
            self.activation_function = activation_function
        self.dropout = dropout
        self.elm_mode = elm_mode

    def forward(self, features, **kwargs):
        x = features
        if not self.elm_mode:
            if self.dropout is not None:
                x = self.dropout(x)
            x = self.input_weights(x)
            x = self.activation_function(x)
            if self.dropout is not None:
                x = self.dropout(x)
            x = self.output_weights(x)
        return x

    def set_elm_mode(self):
        self.elm_mode = True

    def unset_elm_mode(self):
        self.elm_mode = False


class ExtremeFineTuningTrainer(Trainer):
    def __init__(self, debug=False, *args, **kwargs):
        super(ExtremeFineTuningTrainer, self).__init__(*args, **kwargs)
        self.debug = debug #TODO
        self.device = 'cuda' if self.gpus else 'cpu'

        self.model = None
        self.l2_gamma = None
        self.l2_weight = None
        self.input_weights = None

        self.input_size = None
        self.hidden_size = None
        self.output_size = None

        self.input_weights = None
        self.input_bias = None
        self.activation_function = None

        self.output_weights = None
        self.H = None
        self.Gamma = None
        self.Tau = None

    def fit(self, model, train_dataloaders=None, val_dataloaders=None, datamodule=None, ckpt_path=None,
            l2_gamma=1., reset=True, force_hidden_size=False):
        self.model = model.to(self.device)
        self.l2_gamma = l2_gamma

        if reset:
            self.input_size = self.model.classifier.input_weights.in_features
            self.hidden_size = self.model.hidden_size if force_hidden_size else \
                self.model.classifier.input_weights.out_features
            self.output_size = self.model.classifier.output_weights.out_features

            self.input_weights = torch.nn.init.orthogonal_(torch.empty(
                self.hidden_size, self.input_size), gain=1).type(torch.float64).to(self.device)
            self.input_bias = torch.nn.init.uniform_(torch.empty(self.hidden_size)).type(torch.float64).to(self.device)
            self.input_bias = self.input_bias / torch.norm(self.input_bias)
            self.activation_function = self.model.classifier.activation_function

            self.H = None
            self.Gamma = None
            self.Tau = None
            self.n_Gamma = None
            self.n_Tau = None
            self.output_weights = None
            self.l2_weight = 0.1

        start = time.time()

        with torch.no_grad():
            self.model.classifier.set_elm_mode()
            for batch_idx, batch in enumerate(train_dataloaders):
                input_features, label_one_hot = self.model.step_helper(batch)
                self._iterative_elm(input_features.type(torch.float64).to(self.device).T, label_one_hot.T)
                if self.debug:
                    print(f'\rI-ELM Batch#{batch_idx}', end='')

            new_input_weights = torch.nn.Linear(
                in_features=self.input_size,
                out_features=self.hidden_size,
                bias=True,
                device=model.device,
                dtype=model.dtype
            )
            new_input_weights.weight.copy_(self.input_weights)
            new_input_weights.bias.copy_(self.input_bias)

            new_output_weights = torch.nn.Linear(
                in_features=self.hidden_size,
                out_features=self.output_size,
                bias=False,
                device=model.device,
                dtype=model.dtype
            )
            new_output_weights.weight.copy_(self.output_weights)

            model.classifier.input_weights = new_input_weights
            model.classifier.output_weights = new_output_weights

            self.model.classifier.unset_elm_mode()

        end = time.time()
        print(f'\nI-ELM Training Time: {end - start}')

    def _iterative_elm(self, input_features, label_one_hot):
        ones = torch.ones(input_features.shape[1]).to(self.device)
        self.H = (self.input_weights @ input_features + torch.outer(self.input_bias, ones)).type(torch.float32).to(self.device)
        self.H = self.activation_function(self.H).type(torch.float64).to(self.device)
        if self.Gamma is not None or self.Tau is not None:
            self.Gamma += (self.H @ self.H.T)
            self.Tau += (label_one_hot @ self.H.T)
        else:
            self.Gamma = self.H @ self.H.T
            self.Tau = label_one_hot @ self.H.T

        weighted_gamma = (torch.eye(self.H.shape[0], dtype=torch.float64).to(self.device) / self.l2_weight) @ self.Gamma
        inverted_weighted_gamma = np.linalg.inv(weighted_gamma.cpu())
        self.output_weights = self.Tau @ torch.tensor(inverted_weighted_gamma).to(self.device)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

# EFT 학습에 사용할 DataLoader 준비
def prepare_dataloader(embeddings, labels, batch_size):
    labels_tensor = torch.tensor(labels)
    dataset = TensorDataset(embeddings, labels_tensor)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

# EFT 학습 과정
def train_eft_with_combined_embeddings(combined_embeddings, labels, hidden_size, num_labels, lr, epochs, batch_size):
    train_loader = prepare_dataloader(combined_embeddings, labels, batch_size)

    # EFT 모델 설정
    model = ExtremeFineTuning(
        input_size=combined_embeddings.shape[1],
        hidden_size=hidden_size,
        output_size=num_labels,
        activation_function=torch.nn.PReLU(init=0.5),
        dropout=torch.nn.Dropout(p=0.1, inplace=False),
        lr=lr
    )

    # 모델 학습
    trainer = ExtremeFineTuningTrainer(gpus=1)
    trainer.fit(model, train_loader, force_hidden_size=True)
